# Korteweg–De Vries equation

The [Korteweg–De Vries equation](https://en.wikipedia.org/wiki/Korteweg%E2%80%93De_Vries_equation) is defined on 1D space with the following form:

$$
\frac{\partial \phi}{\partial t}
=-c_1\frac{\partial^3 \phi}{\partial x^3} + c_2 \phi\frac{\partial\phi}{\partial x}
$$

where $c_1$ and $c_2$ are constants.

In [1]:
from torchfsm.operator import Operator,Dispersion,Convection

def KortewegDeVries(dispersion_coef=-1,convection_coef:float=6.0) -> Operator:
    return dispersion_coef*Dispersion()+convection_coef*Convection()

# same configuration as the animation in wikipedia
kdv=KortewegDeVries(-0.022**2,-1)

In [2]:
import torch
from torchfsm.mesh import MeshGrid
from torchfsm.plot import plot_traj
from torchfsm.traj_recorder import AutoRecorder,IntervalController, CPURecorder
from torchfsm.field import truncated_fourier_series
device='cuda' if torch.cuda.is_available() else 'cpu'
L=2.0; N=256; 

In [3]:
mesh=MeshGrid([(0,L,N)],device=device)
x=mesh.bc_mesh_grid()
u_0=torch.cos(torch.pi*x)
traj=kdv.integrate(
    u_0=u_0,
    mesh=mesh,
    dt=0.001,
    step=2000,
    trajectory_recorder=AutoRecorder(IntervalController(10))
)

In [4]:
plot_traj(traj)
# check https://en.wikipedia.org/wiki/File:KdV_equation.gif for comparison

Although high-dimensional Korteweg–De Vries equations can be defined, they are not commonly used in practice. Here, we show some examples of how its solution in high dimensions:

## Solution in 2D

In [5]:
mesh=MeshGrid([(0,L,N)]*2,device=device)
u_0=truncated_fourier_series(mesh,freq_threshold=3,n_channel=2)
traj=kdv.integrate(
    u_0=u_0,
    mesh=mesh,
    dt=0.01,
    step=2000,
    trajectory_recorder=AutoRecorder(IntervalController(50))
)

In [6]:
plot_traj(traj)

## Solution in 3D

In [7]:
mesh=MeshGrid([(0,L,N)]*3,device=device)
u_0=truncated_fourier_series(mesh,freq_threshold=3,n_channel=3)
traj=kdv.integrate(
    u_0=u_0,
    mesh=mesh,
    dt=0.01,
    step=200,
    trajectory_recorder=CPURecorder(IntervalController(10)),
    progressive=True
)

Integrating:   0%|          | 0/200 [00:00<?, ?it/s]

In [8]:
plot_traj(traj)